# 랜덤포레스트

- 다수의 의사결정 트리들을 배깅하여 분류 또는 회귀를 수행하는 방식이다.

⭐배깅:같은 종류의 알고리즘 모델을 여러 개 결합하여 예측하는 방법

- 각 트리는 전체 학습 데이터 중 서로 다른 데이터를 샘플링하여 일부 데이터를 제외한 후 최적의 특징을 찾아 트리를 분기한다.

## 알고리즘의 특징

- 비교적 좋은 성능, 과대적합의 위험의 감소
- 트리들이 조금씩 다른 특성을 갖게 되어 일반화 성능을 향상할 수 있다.
- 샘플링하는 과정에서 한 샘플이 중복되어 추출될 수도 있다.
- 기본 매개변수 설정만으로도 좋은 결과를 얻을 수 있다.
- 랜덤 포레스트의 특성 중요도는 각 트리의 특성 중요도를 취합한 것이다.


In [34]:
import numpy as no
import pandas as pd
import sklearn

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [36]:
df= pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/titanic.csv")

In [37]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [39]:
df = df.drop(columns='Cabin',axis=1)
d_mean= df['Age'].mean()
df['Age'].fillna(d_mean,inplace=True)

d_mode = df['Embarked'].mode()[0]
df["Embarked"].fillna(d_mode,inplace=True)
d_mean

<ipython-input-39-d59d20d33a35>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(d_mean,inplace=True)
<ipython-input-39-d59d20d33a35>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.metho

np.float64(29.69911764705882)

In [41]:
from sklearn.preprocessing import LabelEncoder

df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df['Embarked'] = LabelEncoder().fit_transform(df['Embarked'])
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,2
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,2
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",1,27.000000,0,0,211536,13.0000,2
887,888,1,1,"Graham, Miss. Margaret Edith",0,19.000000,0,0,112053,30.0000,2
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",0,29.699118,1,2,W./C. 6607,23.4500,2
889,890,1,1,"Behr, Mr. Karl Howell",1,26.000000,0,0,111369,30.0000,0


In [42]:
df['F']=df['SibSp']+ df["Parch"]

In [43]:
X= df[['Pclass','Sex','F','Embarked','Fare','Age']]
y= df['Survived']

In [44]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [66]:
rf = RandomForestClassifier(n_estimators=100,max_depth=2)
# 50개의 결정트리를 사용한다, 일반적으로 트리의 개수가 많을 수록 안정적인 예측이 가능하지만, 계산비용이 증가한다.
# 각 트리의 깊이를 3으로 제한, 이로 인해 트리가 복잡해지지 않도록 하여 과적합을 방지할 수 있습니다.
rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=2)

In [67]:
pred = rf.predict(X_test)

In [68]:
from sklearn.metrics import accuracy_score
acc= accuracy_score(y_test,pred)
print(acc)

0.7486033519553073
